In [13]:
from google.adk.agents import Agent
from dotenv import load_dotenv
from typing import Dict , List
import sendgrid
import os
from google.adk.tools import agent_tool
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from pydantic import BaseModel , Field


In [14]:
load_dotenv()

True

In [15]:
google_api_key = os.getenv('GOOGLE_API_KEY')
gemini_llm_model = os.getenv('GEMINI_LLM_MODEL')
sendgrid_api_key = os.getenv('SENDGRID_API_KEY')

In [16]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."


In [17]:
sales_agent1 = Agent(
    name="Professional_Sales_Agent",
    model = gemini_llm_model,
    description=("You write professional, serious cold emails"),
    instruction=instructions1
)
sales_agent1_tool = agent_tool.AgentTool(agent=sales_agent1)

sales_agent2 = Agent(
    name = "Engaging_Sales_Agent",
    model = gemini_llm_model,
    description=("You write witty, engaging cold emails that are likely to get a response"),
    instruction=instructions2
)

sales_agent2_tool = agent_tool.AgentTool(agent=sales_agent2)

sales_agent3 = Agent(
    name = "Busy_Sales_Agent",
    model = gemini_llm_model,
    description=("You write concise, to the point cold emails."),
    instruction=instructions2
)

sales_agent3_tool = agent_tool.AgentTool(agent=sales_agent3)



In [18]:
best_email_picker_instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only."

best_email_picker = Agent(
    name="sales_picker",
    description=("You pick the best cold sales email from a set of provided emails"),
    instruction=(best_email_picker_instructions),
    model=gemini_llm_model,
)

best_email_picker_agent_tool = agent_tool.AgentTool(agent=best_email_picker)

In [19]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

subject_writer = Agent(
    name="Email_subject_writer", 
    description=("You write the subject of the email based on the email body provided"),
    instruction=(subject_instructions), 
    model=gemini_llm_model
    )

subject_writer_agent_tool = agent_tool.AgentTool(agent=subject_writer)

In [20]:
html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

html_converter = Agent(
    name="HTML_email_body_converter", 
    description=("You convert the email into HTML format"),
    instruction=(html_instructions), 
    model=gemini_llm_model
    )

html_converter_agent_tool = agent_tool.AgentTool(agent=html_converter)

In [21]:
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = os.getenv("FROM_EMAIL")
    to_email = os.getenv("TO_EMAIL")
    from_email = Email(from_email)  # Change to your verified sender
    to_email = To(to_email)  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [22]:
tools = [
    sales_agent1_tool, 
    sales_agent2_tool, 
    sales_agent3_tool, 
    best_email_picker_agent_tool,
    subject_writer_agent_tool,
    html_converter_agent_tool,
    send_html_email]

tools

 <function __main__.send_html_email(subject: str, html_body: str) -> Dict[str, str]>]

In [23]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Use the email_picker_agent_tool to Review the drafts and choose the single best email using your judgment of which one is most effective.

3. Create the subject line: Use the subject_writer_agent_tool to write the subject of the email chosen.

4. Convert to html email format: Use the html_converter_agent_tool to convert the email to html format
 
5. Use the send_html_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""

sales_manager = Agent(
    name="Sales_Manager", 
    instruction=(sales_manager_instructions), 
    description=("Your goal is to find the single best cold sales email using the sales_agent tools, pick the best email, get the subject of the email, convert to html format and send the email"),
    tools=tools, 
    model=gemini_llm_model,
    )

In [32]:
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
import asyncio

APP_NAME = "Cold_Email_App"
USER_ID = "user_1"
SESSION_ID = "session_001"  # Using a fixed ID for simplicity

async def main():
    session_service = InMemorySessionService()

    # Create the specific session where the conversation will happen
    session = await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )
    print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")


In [38]:
runner = Runner(
    agent=sales_manager, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Runner created for agent 'Sales_Manager'.


In [34]:
from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

In [ ]:
async def run_conversation():
    message = "Send out a cold sales email addressed to Dear CEO from Alice"
    await call_agent_async(
        message,
        runner=runner,
        user_id=USER_ID,
        session_id=SESSION_ID
        )

In [ ]:
await run_conversation()